In [ ]:
%pip install --quiet langchain langchain-openai langchain-experimental aperturedb openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.3/414.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import json
from google.colab import userdata
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from typing import List, Dict
from aperturedb.Utils import Utils
from langchain_community.vectorstores import ApertureDB
from aperturedb.CommonLibrary import create_connector
from pydantic import BaseModel
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_KEY')

## Connection to ApertureDB

In [ ]:
!adb config create --active --from-json

In [ ]:
# Create the connector for ApertureDB
client = create_connector()

# Use the connector to create a Utils object and print the summary
utils = Utils(client)
utils.summary()

In [ ]:
query = [{
    "GetStatus": {
    }
}]

# Execute the query to get back a JSON response for GetStatus
response, blobs = client.query(query)

client.print_last_response()

## Dataset

In [ ]:
data = [{'Id': '1',
  'content': 'Example Corp, the US-based maker of the popular Widget personal gizmo, has recently extended its worldwide distribution channels by partnering with AnyCompany Logistics, an international shipping, storage, and last-mile distribution provider. Widget is an AI-augmented personal desktop pet whose conversational capabilities are powered by a new breed of generative AI technologies. Developed in the Austin-based Example Corp labs, the toys are manufactured in Taiwan.',
  'class': 'Article',
  'publish_date': '2023-07-01',
  'author': 'Example Corp',
  'relation': ['3']},
 {'Id': '2',
  'content': 'The UK’s Christmas top-10 toy predictions are already in, even though it’s only August, with industry analysts predicting huge demand for Example Corp’s Widget chatty desktop pet. Retailers in London, Manchester, and other major cities have already placed orders for over 1 million units, to the value of $15 million, and those numbers are only set to increase in the months leading up to Christmas.',
  'class': 'Article',
  'publish_date': '2023-08-01',
  'author': 'Toy Industry Analyst',
  'relation': ['4']},
 {'Id': '3',
  'content': 'AnyCompany Logistics today announced that it is cutting shipping times from Taiwan to the UK from 2 weeks to just 3 days by directing all distribution through the recently opened Fictitious Canal. The Fictitious Canal is set to become one of the most heavily trafficked waterways over the next 5 years as manufacturers and distributors look to move from air to sea to cut energy spend and save costs.',
  'class': 'Article',
  'publish_date': '2023-07-15',
  'author': 'AnyCompany Logistics',
  'relation': ['4']},
 {'Id': '4',
  'content': 'The Fictitious Canal continues to experience blockages and delays following landslips. Over 5 million tons of rock has now blocked the southern basin, trapping container vessels and disrupting international supply chains. Goods destined for Europe are expected to be delayed for several months, meaning that Christmas inventory for many retailers is under threat.',
  'class': 'Article',
  'publish_date': '2023-07-20',
  'author': 'Shipping News',
  'relation': []}]

## Ingestion

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
def convert_to_langchain_docs(data: List[Dict]) -> List[Document]:
    """
    Convert a list of dictionaries containing article data into LangChain Document objects.

    Args:
        data (List[Dict]): List of dictionaries containing article details.
            Expected keys: "Id", "content", "publish_date", "author", and optionally "relation".
        doc_ids (Dict[str, str]): Mapping of article IDs to UUIDs.

    Returns:
        List[Document]: List of LangChain Document objects.
    """
    documents = []
    for item in data:
        metadata = {
            "id": item.get("Id"),\
            "author": item.get("author"),
            "published_at": item.get("publish_date")
        }
        doc = Document(page_content=item.get("content", ""), metadata=metadata)
        documents.append(doc)

    return documents

In [ ]:
docs = convert_to_langchain_docs(data)

In [ ]:
docs

[Document(metadata={'id': '1', 'author': 'Example Corp', 'published_at': '2023-07-01'}, page_content='Example Corp, the US-based maker of the popular Widget personal gizmo, has recently extended its worldwide distribution channels by partnering with AnyCompany Logistics, an international shipping, storage, and last-mile distribution provider. Widget is an AI-augmented personal desktop pet whose conversational capabilities are powered by a new breed of generative AI technologies. Developed in the Austin-based Example Corp labs, the toys are manufactured in Taiwan.'),
 Document(metadata={'id': '2', 'author': 'Toy Industry Analyst', 'published_at': '2023-08-01'}, page_content='The UK’s Christmas top-10 toy predictions are already in, even though it’s only August, with industry analysts predicting huge demand for Example Corp’s Widget chatty desktop pet. Retailers in London, Manchester, and other major cities have already placed orders for over 1 million units, to the value of $15 million,

In [ ]:
vector_db = ApertureDB.from_documents(docs, embeddings)

In [ ]:
def add_articles_to_db(client, articles):
    query = []

    # Add each article as an entity.
    for article in articles:
        query.append({
            "AddEntity": {
                "_ref": int(article["Id"][0]),
                "class": "Article",
                "properties": {
                    "Id": int(article["Id"][0]),
                    "content": article["content"],
                    "publish_date": article["publish_date"],
                    "author": article["author"]
                },
                "if_not_found": {
                    "Id": ["==", int(article["Id"][0])]
                }
            }
        })

    # Create connections based on the relationship field.
    for article in articles:
        for dst in article.get("relation", []):
            print(dst)
            query.append({
                "AddConnection": {
                    "class": "relationship",
                    "src": int(article["Id"][0]),
                    "dst": int(dst),
                    "properties": {"connected_to": int(dst)}
                }
            })

    # Execute the query and print the last response.
    response, blobs = client.query(query)
    client.print_last_response()
    return response, blobs

In [ ]:
from langchain_community.vectorstores import ApertureDB
vector_db = ApertureDB.from_documents(docs, embeddings)

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
context = retriever.get_relevant_documents("What are the sales prospects for Example Corp in the UK?")

In [ ]:
context

[Document(id='956cedfb-d437-49e3-804a-052a6875cf1f', metadata={'author': 'Toy Industry Analyst', 'id': '2', 'published_at': '2023-08-01'}, page_content='The UK’s Christmas top-10 toy predictions are already in, even though it’s only August, with industry analysts predicting huge demand for Example Corp’s Widget chatty desktop pet. Retailers in London, Manchester, and other major cities have already placed orders for over 1 million units, to the value of $15 million, and those numbers are only set to increase in the months leading up to Christmas.'),
 Document(id='bd617b30-e58c-4603-9de5-259d335f2b0c', metadata={'author': 'Toy Industry Analyst', 'id': '2', 'published_at': '2023-08-01'}, page_content='The UK’s Christmas top-10 toy predictions are already in, even though it’s only August, with industry analysts predicting huge demand for Example Corp’s Widget chatty desktop pet. Retailers in London, Manchester, and other major cities have already placed orders for over 1 million units, to

In [ ]:
def retrieve_with_multi_hop(client, query):
    """
    Retrieves an article ID from the vector database and fetches all connected articles using multi-hop retrieval.

    Args:
        client: The API client used to execute queries.
        query: The search query for initial document retrieval.

    Returns:
        A list of relevant article contents.
    """
    retrieved_docs = retriever.get_relevant_documents(query)
    if not retrieved_docs:
        return []

    start_id = retrieved_docs[0].metadata.get("Id")

    return multi_hop_article_content(client, start_id)

In [ ]:
def multi_hop_article_content(client, start_id):
    """
    Given a starting article id, fetches its content and then hops along the
    'connected_to' chain until no further connection is found.

    Args:
        client: The API client used to execute queries.
        start_id: The starting Article Id.

    Returns:
        A list of article content strings gathered from the hop chain.
    """
    context = []       # List to hold the content of each Article
    visited = set()    # To avoid loops
    current_id = start_id

    while current_id is not None and current_id not in visited:
        visited.add(current_id)

        # --- Step 1: Fetch the Article content ---
        article_query = [{
            "FindEntity": {
                "with_class": "Article",
                "_ref": 1,
                "constraints": {
                    "Id": ["==", current_id]
                },
                "results": {
                    "list": ["Id", "content"]
                }
            }
        }]
        response, blobs = client.query(article_query)
        article_result = response[0]["FindEntity"]
        if article_result["returned"] == 0:
            # No article found; exit the loop.
            break

        article = article_result["entities"][0]
        content = article.get("content", "")
        context.append(content)

        # --- Step 2: Find the next connection ---
        connection_query = [{
            "FindEntity": {
                "with_class": "Article",
                "_ref": 1,
                "constraints": {
                    "Id": ["==", current_id]
                }
            }
        },{
            "FindConnection": {
                "with_class": "relationship",
                "src": 1,
                "results": {
                    "list": ["connected_to"],
                }
            }
        }]
        response, blobs = client.query(connection_query)
        connection_result = response[1]["FindConnection"]
        connections = connection_result.get("connections", [])

        # Look for the first non-null 'connected_to' value.
        next_id = None
        for conn in connections:
            if conn.get("connected_to") is not None:
                next_id = conn["connected_to"]
                break

        # If no valid next connection, end the loop.
        current_id = next_id

    return context

{'entities': [{'Id': 1, 'content': 'Example Corp, the US-based maker of the popular Widget personal gizmo, has recently extended its worldwide distribution channels by partnering with AnyCompany Logistics, an international shipping, storage, and last-mile distribution provider. Widget is an AI-augmented personal desktop pet whose conversational capabilities are powered by a new breed of generative AI technologies. Developed in the Austin-based Example Corp labs, the toys are manufactured in Taiwan.'}], 'returned': 1, 'status': 0}
{'entities': [{'Id': 3, 'content': 'AnyCompany Logistics today announced that it is cutting shipping times from Taiwan to the UK from 2 weeks to just 3 days by directing all distribution through the recently opened Fictitious Canal. The Fictitious Canal is set to become one of the most heavily trafficked waterways over the next 5 years as manufacturers and distributors look to move from air to sea to cut energy spend and save costs.'}], 'returned': 1, 'status'

In [ ]:
def query_with_context(context, user_query, model="gpt-4o"):
    """
    Uses the gathered context and a user query to generate a response using OpenAI's API.

    Args:
        context (list of str): List of article contents from the multi-hop process.
        user_query (str): The query to ask the assistant.
        model (str): The model to use for chat completions.

    Returns:
        The response message from the assistant.
    """
    client = OpenAI()

    # Combine the context list into a single string. Separate each entry with two newlines

    # Construct the messages for the chat completion.
    messages = [
        {"role": "developer", "content": "You are a helpful assistant."},
        {"role": "system", "content": f"Here is some context gathered from our articles:\n\n{context}"},
        {"role": "user", "content": user_query}
    ]

    completion = client.chat.completions.create(
        model=model,
        messages=messages
    )

    return completion.choices[0].message

## Graph RAG

In [ ]:
query = "What are the sales prospects for Example Corp in the UK?"
starting_context = retrieve_with_multi_hop(client, query)
query_with_context(starting_context, query, model="gpt-4o")

ChatCompletionMessage(content="The sales prospects for Example Corp in the UK may currently face challenges due to supply chain disruptions. While Example Corp recently partnered with AnyCompany Logistics to expand its worldwide distribution channels and reduce shipping times from Taiwan to the UK from 2 weeks to just 3 days, these plans have been affected by blockages and delays in the Fictitious Canal. The canal is experiencing landslips, which have trapped container vessels and disrupted supply chains. As a result, goods destined for Europe, including Example Corp's products, could face significant delays, potentially threatening inventory levels for UK retailers, especially during the critical holiday season. This disruption could impact sales negatively in the short term if Example Corp is unable to meet demand due to these shipping issues.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

## Vanilla RAG

In [ ]:
query_with_context(context, "What are the sales prospects for Example Corp in the UK?", model="gpt-4o")

ChatCompletionMessage(content='The sales prospects for Example Corp in the UK look quite promising. According to industry analysts, there is a predicted huge demand for Example Corp’s Widget chatty desktop pet. Retailers in major UK cities like London and Manchester have already placed significant orders for over 1 million units, valued at $15 million. These numbers are expected to increase as the Christmas season approaches, indicating strong sales prospects for Example Corp in this market.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)